### Dépendances

In [1]:
import urllib
import bs4
import pandas as pd
from urllib import request
import numpy as np
import re

# Dépendances wikipedia
import wikipedia
from wikipedia import WikipediaPage
wikipedia.set_lang("fr")

# Dépendances traitement de texte
import spacy
import nltk
nlp = spacy.load("fr_core_news_sm")
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=stopwords.words('french')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\proui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Objectif final:** créer le dataframe composé des auteurs et de leur mouvement littéraire.

### Scraping de la page recensant les mouvements littéraires

Voici la [page](https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_par_mouvement_ou_courant_litt%C3%A9raire) que nous allons scraper afin d'obtenir les mouvements littéraires.<br>


In [2]:
mvm_url="https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_par_mouvement_ou_courant_litt%C3%A9raire"
mvm_request=request.urlopen(mvm_url).read()
mvm_page = bs4.BeautifulSoup(mvm_request, "lxml") #qui est bien une page html maintenant 

<code>mvm_blayer</code> ; on identifie le bloc vert regroupant tous les mouvements avec <code>id</code>.<br> 
Son premier élément est HS, on le retire.<br>
#
![title](ress/image_mvm_page.png)

In [3]:
mvm_blayer=mvm_page.find("div",{"id":"mw-subcategories"}).findAll("ul")
mvm_blayer=mvm_blayer[1::]

<code>mvm_list</code> liste issue du scraping de la page précédente : composée de l'ensemble des mouvements littéraires de la forme 'Catégorie:XXX'. Cette forme sera utile pour créer les URLs.<br>
<code>data_mvm</code> on retire 'Catégorie:' (pas la forme la plus optimisée pour le dataset mais pratique pour comparer lors de l'implémentation du scraping.

In [6]:
mvm_list=[]
for ul in mvm_blayer:
    interli=[]
    listli=ul.findAll("li")
    for li in listli:
        try:
            interli.append(li.find("a").get("title"))
        except:
            pass
    mvm_list.append(interli)
#print(mvm_list)

# Liste des mouvements qui sera utilisée pour créer le dataframe 
data_mvm=[]
for l in mvm_list:
    for el in l:
        pep=re.sub('Catégorie:','',el)
        data_mvm.append(pep)
print('Liste des différents mouvements: ', data_mvm)

Liste des différents mouvements:  ['Écrivain de la Beat Generation', 'Écrivain de la Génération perdue', 'Écrivain du courant des Hussards', 'Écrivain jungien', 'Écrivain du courant réaliste', 'Écrivain régionaliste', 'Écrivain de la Renaixença', 'Écrivain romantique', 'Écrivain surréaliste', 'Écrivain symboliste']


On modifie <code>mvm_list</code> afin d'être utilisable pour les URLs simplement en concaténant une base fixe et ses éléments.

In [8]:
def modifL(list):
    L=[]
    for i in list:
        for j in i:
            pep=re.sub(' ','_',j)               # on remplace les espaces 
            pep=pep.replace('é','%C3%A9')       # corrections pour l'url
            pep=pep.replace('É','%C3%89')           
            pep=pep.replace('ç','%C3%A7')
            L.append(pep)   
    return(L)

mvm_list=modifL(mvm_list)
print(mvm_list)

['Cat%C3%A9gorie:%C3%89crivain_de_la_Beat_Generation', 'Cat%C3%A9gorie:%C3%89crivain_de_la_G%C3%A9n%C3%A9ration_perdue', 'Cat%C3%A9gorie:%C3%89crivain_du_courant_des_Hussards', 'Cat%C3%A9gorie:%C3%89crivain_jungien', 'Cat%C3%A9gorie:%C3%89crivain_du_courant_r%C3%A9aliste', 'Cat%C3%A9gorie:%C3%89crivain_r%C3%A9gionaliste', 'Cat%C3%A9gorie:%C3%89crivain_de_la_Renaixen%C3%A7a', 'Cat%C3%A9gorie:%C3%89crivain_romantique', 'Cat%C3%A9gorie:%C3%89crivain_surr%C3%A9aliste', 'Cat%C3%A9gorie:%C3%89crivain_symboliste']


### Scraping liste d'auteurs par mouvement 

A première vue on a deux types de page :
- Celles avec une partie "sous-catégorie" puis le bloc avec les écrivains
- Celles sans "sous-catégorie"<br>
#
![title](ress/image_beatgen.png)
#
![title](ress/image_genperdue.png)

**Solution** : passer par l'identification <code>.find("div",{"id":'mw-pages'})</code>

Maintenant qu'on a la liste des différents mouvements on veut une liste d'auteurs par mouvement.<br>
On va scraper les pages associées à ces mouvements de la forme <code>https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:XXX</code> (où XXX est un mouvement). 

In [10]:
# == Création des URLs ==
wikibase="https://fr.wikipedia.org/wiki/"

url_list=[wikibase+cat for cat in mvm_list]
print('URL pour les écrivains de la Beat Generation: ',url_list[0])

URL pour les écrivains de la Beat Generation:  https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_de_la_Beat_Generation


<code>aut_crap</code> fonction qui prend en argument un URL et retourne la liste des auteurs présents sur cette page.

In [11]:
def aut_scrap(url):

    request_text= request.urlopen(url).read()
    page = bs4.BeautifulSoup(request_text, "lxml") #qui est bien une page html maintenant 

    # == Obtenir toutes les catégories par lettre ==
    base=page.find("div",{"id":'mw-pages'}).findAll("ul")
    base=base[1::]

    # == Obtenir les auteurs pour tous les "blocs" précédents ==
    aut_list=[]
    auteurs1=[]
    for ul in base:
        liste_li=ul.findAll("li")
        for li in liste_li:
            try:
                auteurs1.append(li.find("a").get("title"))
            except:
                pass
        
    return(auteurs1)

<code>aut1</code> liste des auteurs groupés par mouvements.

In [14]:
aut1=[aut_scrap(url) for url in url_list]
aut1[:2]    

[['William S. Burroughs',
  'Lucien Carr',
  'Carolyn Cassady',
  'John Clellon Holmes',
  'Gregory Corso',
  'Lenore Kandel',
  'Jack Kerouac',
  'Jan Kerouac',
  'Gerald Nicosia',
  'Peter Orlovsky',
  'John Wieners'],
 ['Sherwood Anderson',
  'Sylvia Beach',
  'E. E. Cummings',
  'John Dos Passos',
  'T. S. Eliot',
  'F. Scott Fitzgerald',
  'Ernest Hemingway',
  'Archibald MacLeish',
  'Ezra Pound',
  'Gertrude Stein',
  'John Steinbeck']]

<code>data_mvm2</code> liste de la même "forme" (même ordre, même longueur...) que <code>aut1</code> qui associe donc à chaque écrivain son mouvement.

In [15]:
data_mvm2=[]
for i in range(len(aut1)):
    inter=[]
    for j in range(len(aut1[i])):
        inter.append(data_mvm[i])
    data_mvm2.append(inter)
data_mvm2[:2]

[['Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation'],
 ['Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue']]

### Création du dataframe

In [16]:
xdata, ydata=[],[]

for gr_aut in aut1:
    for aut in gr_aut:
        xdata.append(aut)
for gr_mvm in data_mvm2:
    for mvm in gr_mvm:
        ydata.append(mvm)

data={'Nom':xdata, 'Mouvement':ydata}
df=pd.DataFrame(data)
print(df)

                      Nom                       Mouvement
0    William S. Burroughs  Écrivain de la Beat Generation
1             Lucien Carr  Écrivain de la Beat Generation
2         Carolyn Cassady  Écrivain de la Beat Generation
3     John Clellon Holmes  Écrivain de la Beat Generation
4           Gregory Corso  Écrivain de la Beat Generation
..                    ...                             ...
256     Maurice Beaubourg             Écrivain symboliste
257        Otokar Březina             Écrivain symboliste
258   Emanuil Popdimitrov             Écrivain symboliste
259         Marcel Schwob             Écrivain symboliste
260     Friedebert Tuglas             Écrivain symboliste

[261 rows x 2 columns]


On nettoie la colonne <code>Mouvement</code> pour la labelisation à suivre.

In [17]:
clean_words=['Écrivain de la ', 'Écrivain du courant des ', 'Écrivain ', 'Écrivain du courant ']
def mvm_clean(word):
    for char in clean_words:
        word=word.replace(char,"")
    return word

df['Mouvement']=df['Mouvement'].apply(lambda x : mvm_clean(x))

In [18]:
df.head()

,Nom,Mouvement
0,William S. Burroughs,Beat Generation
1,Lucien Carr,Beat Generation
2,Carolyn Cassady,Beat Generation
3,John Clellon Holmes,Beat Generation
4,Gregory Corso,Beat Generation


### Scraping du summary

(ligne suivante assez longue à exécuter ~3/4 minutes)

In [16]:
sum_aut=[WikipediaPage(aut).summary for aut in df['Nom']]
df['summary']=sum_aut

Text processing : 
- nettoyage 
- lemmatisation et suppression des stop words

In [17]:
#1. Enlève les '\n' qui se trouvent dans certains textes
def cleanN(text): 
    words=text.split()
    for word in words :     #on a des \n seuls ou "dans" des mots 
        if word=='\n':
            words.remove(word)         
        elif '\n'in word:
            word.replace('\n',' ')
        else:
            pass
    return(' '.join(words))

#2. Supprime les stop words / lemmatisation
def clean_txt(s):
    doc = nlp(s)
    L=[]
    for token in doc:
        s_t = str(token)
        alphanum=True
        if not s_t in stop:
            for char in s_t:
                alphanum = char.isalnum() and alphanum
            if alphanum:            
                L.append(token.lemma_)
    L=np.array(L)
    return ' '.join(L)

On ajoute les colonnes suivantes au dataset :
- <code>sum_lem</code> summaries nettoyés et lemmatisés 
- <code>mvm_label</code> on labelise les mouvements pour la classification

In [18]:
#sum_lem
sum_lem=[clean_txt(cleanN(sum)) for sum in df['summary']]   
df['sum_lem']=sum_lem

In [19]:
#mvm_label
from sklearn import preprocessing
LE=preprocessing.LabelEncoder()
mvm_label=LE.fit_transform(df['Mouvement'])
df['mvm_label']=mvm_label

In [22]:
df[15:27]

,Nom,Mouvement,summary,sum_lem,mvm_label
15,T. S. Eliot,Génération perdue,"T. S. Eliot, de son nom complet Thomas Stearns...",Eliot nom complet Thomas Stearns Eliot 26 sept...,1
16,F. Scott Fitzgerald,Génération perdue,"Francis Scott Key Fitzgerald, né le 24 septemb...",Francis Scott Key fitzgerald naître 24 septemb...,1
17,Ernest Hemingway,Génération perdue,"Ernest Hemingway [ˈɝnɪst ˈhɛmɪŋˌweɪ], né le 21...",Ernest Hemingway ˈɝnɪst ˈhɛmɪŋˌweɪ naître 21 j...,1
18,Archibald MacLeish,Génération perdue,"Archibald MacLeish (1892-1982) est un poète, d...",Archibald MacLeish 1892 1982 poète dramaturge ...,1
19,Ezra Pound,Génération perdue,"Ezra Weston Loomis Pound (né à Hailey, alors d...",Ezra Weston Loomis Pound naître Hailey alors t...,1
20,Gertrude Stein,Génération perdue,"Gertrude Stein, née le 3 février 1874 à Allegh...",Gertrude Stein naître 3 février 1874 Allegheny...,1
21,John Steinbeck,Génération perdue,"John Ernst Steinbeck, Jr. (en anglais : [ˈstaɪ...",John Ernst Steinbeck Jr anglais ˈstaɪnbɛk naît...,1
22,Antoine Blondin,Hussards,"Antoine Blondin, né le 11 avril 1922 à Paris e...",Antoine Blondin naître 11 avril 1922 Paris mor...,2
23,Jean-François Coulomb,Hussards,Jean-François Coulomb (1956) est un journalist...,Coulomb 1956 journaliste producteur écrivain f...,2
24,Michel Déon,Hussards,"Michel Déon, né Édouard Michel le 4 août 1919 ...",Michel Déon naître Édouard Michel 4 août 1919 ...,2


In [21]:
#compression_opts = dict(method='zip', archive_name='out.csv') 
#df.to_csv('out.zip', index=False,compression=compression_opts)  

df.to_csv('data.csv')